In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import os
import json

# Function to update genres in the dataframe
def update_genres_for_artist(spotify_client, artist_name, dataframe):
    try:
        results = spotify_client.search(q='artist:' + artist_name, type='artist')
        if results['artists']['items']:
            artist = results['artists']['items'][0]
            spotify_genres = artist['genres']
            for genre in spotify_genres:
                if genre in dataframe.columns:
                    dataframe.loc[dataframe['Artist'] == artist_name, genre] = 1
    except Exception as e:
        print(f"Error with artist {artist_name}: {e}")

# Spotify API credentials
client_id = os.getenv('SPOTIFY_CLIENT_ID')
client_secret = os.getenv('SPOTIFY_CLIENT_SECRET')
redirect_uri = 'REDIRECT_URI'

# Cache path
username = os.getlogin()
cache_path = os.path.join(os.path.expanduser('~'), 'your_app_cache')

auth_manager = SpotifyOAuth(client_id=client_id,
                            client_secret=client_secret,
                            redirect_uri=redirect_uri,
                            cache_path=cache_path,
                            scope="user-library-read")

sp = spotipy.Spotify(auth_manager=auth_manager)

try:
    token_info = auth_manager.get_cached_token()
    if not token_info:
        code = input("Enter the code from the URL: ")
        token_info = auth_manager.get_access_token(code, as_dict=True)
        if isinstance(token_info, str):
            token_info = json.loads(token_info)
    sp.auth = token_info['access_token']
    print("Authentication Successful.")
except Exception as e:
    print("Error during Authentication:", e)

# Load the existing CSV file
file_path = '/path/to/your/file'
df = pd.read_csv(file_path)

# Update the genre information for each artist in the dataframe
for artist in df['Artist'].dropna().unique():
    update_genres_for_artist(sp, artist, df)

# Save the updated dataframe back to the CSV file
df.to_csv(file_path, index=False)

print("CSV file updated with new genre information.")
